In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv('train.csv')
df.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
417,418,1,2,"Silven, Miss. Lyyli Karoliina",female,18.0,0,2,250652,13.000,NaN,S
777,778,1,3,"Emanuel, Miss. Virginia Ethel",female,5.0,0,0,364516,12.475,NaN,S
211,212,1,2,"Cameron, Miss. Clear Annie",female,35.0,0,0,F.C.C. 13528,21.000,NaN,S


In [4]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [5]:
# train_test_split the model
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                    df['Survived'], test_size=0.2, random_state=42) 

In [6]:
X_train.sample(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
444,3,male,NaN,0,0,8.1125,S
62,1,male,45.0,1,0,83.4750,S


In [7]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough') 

In [8]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
], remainder='passthrough') 
# not doing drop='first' causing using DecisionTreeClassifier 

In [9]:
# scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0, 10)) # slice suggest all columns b/w range 0, 10
]) 

In [10]:
# feature selection
trf4 = SelectKBest(score_func=chi2, k=5) 

In [11]:
# train the model
trf5 = DecisionTreeClassifier() 

## **Create Pipeline**

In [12]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

### **Pipeline VS make_pipeline**
Pipeline requires naming of steps, make_piepline does not.
(same applies to ColumnTransformer VS make_column_transformer)

In [13]:
# Alternate Syntax
pipe2 = make_pipeline(trf1, trf2, trf3, trf4, trf5) 

In [14]:
# train the pipeline
pipe.fit(X_train, y_train) 

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7f8dfcbf8180>)),
                ('trf5', DecisionTreeClassifier())])

In [15]:
# Predict
y_pred = pipe.predict(X_test) 

In [16]:
# check accuracy score
from sklearn.metrics import accuracy_score
print('Accuracy Score:', accuracy_score(y_test, y_pred)) 

Accuracy Score: 0.6256983240223464


## **Explore the Pipeline**

In [17]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=5, score_func=<function chi2 at 0x7f8dfcbf8180>),
 'trf5': DecisionTreeClassifier()}

In [24]:
# to extract value of mean age
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

## **Cross validation using Pipeline**

In [27]:
from sklearn.model_selection import cross_val_score
print('Cross validation score:', cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean())

Cross validation score: 0.6391214419383433
